In [1]:
import pandas as pd
from datasets import load_dataset  # Requires transformers>=4.51.0
from sentence_transformers import SentenceTransformer

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = load_dataset("rohitsaxena/MovieSum")

Repo card metadata block was not found. Setting CardData to empty.


In [3]:
train = ds["train"]
validation = ds["validation"]
test = ds["test"]
dftr = pd.DataFrame(train)
dfva = pd.DataFrame(validation)
dfte = pd.DataFrame(test)

In [4]:
df = pd.concat([dftr, dfva, dfte])
df

,movie_name,imdb_id,script,summary
0,8MM_1999,tt0134273,<script>\n <scene>\n <stage_direction>INT....,Private investigator Tom Welles is contacted b...
1,The Iron Lady_2011,tt1007029,<script>\n <scene>\n <stage_direction>INT....,"In flashbacks, the audience is shown a young M..."
2,Adventureland_2009,tt1091722,<script>\n <scene>\n <scene_description>Ad...,"In 1987, James Brennan plans to have a summer ..."
3,Napoleon_2023,tt13287846,<script>\n <scene>\n <character>NAPOLEON</...,"In 1793, amid the French Revolution, young arm..."
4,Kubo and the Two Strings_2016,tt4302938,<script>\n <scene>\n <character>KUBO</char...,"In feudal Japan, a 12-year-old boy with only o..."
...,...,...,...,...
195,Terminator Salvation_2009,tt0438488,<script>\n <scene>\n <stage_direction>INT....,"In 2003, Dr. Serena Kogan of Cyberdyne Systems..."
196,Collateral Damage_2002,tt0233469,<script>\n <scene>\n <stage_direction>INT....,A bomb detonates in the plaza of the Colombian...
197,Wonder Woman_2017,tt0451279,<script>\n <scene>\n <scene_description>FA...,"In present-day Paris, Diana Prince receives a ..."
198,Flatliners_2017,tt2039338,<script>\n <scene>\n <character>FLATLINERS...,Courtney is a medical student who is obsessed ...


In [5]:
df2 = df.reset_index().drop("index", axis=1)
df2.head(30)

,movie_name,imdb_id,script,summary
0,8MM_1999,tt0134273,<script>\n <scene>\n <stage_direction>INT....,Private investigator Tom Welles is contacted b...
1,The Iron Lady_2011,tt1007029,<script>\n <scene>\n <stage_direction>INT....,"In flashbacks, the audience is shown a young M..."
2,Adventureland_2009,tt1091722,<script>\n <scene>\n <scene_description>Ad...,"In 1987, James Brennan plans to have a summer ..."
3,Napoleon_2023,tt13287846,<script>\n <scene>\n <character>NAPOLEON</...,"In 1793, amid the French Revolution, young arm..."
4,Kubo and the Two Strings_2016,tt4302938,<script>\n <scene>\n <character>KUBO</char...,"In feudal Japan, a 12-year-old boy with only o..."
5,The Woman King_2022,tt8093700,<script>\n <scene>\n <character>THE WOMAN ...,In the West African kingdom of Dahomey in 1823...
6,What They Had_2018,tt6662736,<script>\n <scene>\n <character>WHAT THEY ...,When Alzheimer's-stricken Ruth Everhardt wande...
7,"Synecdoche, New York_2008",tt0383028,<script>\n <scene>\n <scene_description>SY...,Theater director Caden Cotard finds his life u...
8,Black Christmas_2006,tt0454082,<script>\n <scene>\n <scene_description>BI...,"Billy Lenz is born in 1970, with severe jaundi..."
9,Superbad_2007,tt0829482,<script>\n <scene>\n <stage_direction>SUPE...,Seth and Evan are two high school seniors who ...


In [6]:
# Load the model
model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")  # maybe try reranker as well?

# We recommend enabling flash_attention_2 for better acceleration and memory saving,
# together with setting `padding_side` to "left":
# model = SentenceTransformer(
#     "Qwen/Qwen3-Embedding-0.6B",
#     model_kwargs={"attn_implementation": "flash_attention_2", "device_map": "auto"},
#     tokenizer_kwargs={"padding_side": "left"},
# )


In [7]:
MODELPROMPT = "Instruct: Given a movie search query, retrieve relevant movies that match the query in mood, genre, and plot\nQuery:"

In [8]:
# The queries and documents to embed
queries = [
    "I want a sci-fi movie where Earth is destroyed and other planets are explored. It should involve wormholes and time dilation. The main character should use black holes to communicate with the past.",
    "I want a sci-fi movie where the world is a simulation. The main character is a programmer who one day is presented with a choice to remain in the simulation or break free from it.",
    "Private investigator Tom Welles is contacted by Daniel Longdale, attorney for wealthy widow Mrs. Christian. While clearing out her recently deceased husband's safe, she and Longdale find an 8mm movie which appears to depict a real murder of a girl, but Mrs. Christian wants to know for certain. Welles is instructed by Longdale and Mrs. Christian not to reproduce the film in any way, and to keep the investigation secret. After looking through missing persons files, Welles discovers the girl is Mary Ann Mathews and visits her mother, Janet, in North Carolina. She allows Welles to search her house, and he finds Mary Ann's diary, in which she says she relocated to Hollywood to become a film star. Mrs. Mathews demands that he find out what happened to her daughter. In Hollywood, aided by adult video store clerk Max California, Welles infiltrates the world of underground fetish pornography. He attempts to find out if snuff films are real, or if anyone was connected to this film, but finds no evidence that a snuff film industry exists. Contact with sleazy talent scout Eddie Poole leads Welles and Max to director Dino Velvet; Velvet's violently pornographic films often feature a masked man named \"Machine\", who not only brutalizes and tortures women, but also resembles the man featured in Christian's snuff film. To gain more evidence, Welles and California travel to New York and pose as clients interested in commissioning a hardcore BDSM film to be directed by Velvet and starring Machine. Velvet appears to agree, and schedules a meeting. The meeting turns out to be an ambush, as Longdale and Poole appear and hold Welles at gunpoint. The film is revealed as authentic; Mr. Christian contacted Longdale to procure a snuff film, and being unable to find one, Longdale commissioned Velvet and Poole to make one. Velvet and Machine produce a bound and beaten Max, whom they abducted to force Welles to bring them the only surviving copy of the film. As Longdale and Welles go to Welles's car to retrieve the film, Longdale admits he never thought Welles would get as far as he did, and just wanted to placate Mrs. Christian with the investigation. Once Welles delivers the film, Longdale and Velvet burn it and kill Max. As they are about to kill Welles, he confesses Mr. Christian paid $1 million for the film; Velvet, Poole, and Machine received $50,000 and Longdale kept the major portion. In an ensuing fight, Velvet and Longdale are killed; Welles wounds Machine and escapes. Subsequently, he informs Mrs. Christian regarding his findings and recommends she contact the police. Arriving at her estate, he is shocked to hear that she committed suicide immediately after receiving the phone call and notices envelopes that she left behind both for the Mathews family and for him; his envelope contains the rest of his payment and a note reading, \"Try to forget us.\" He instructs his wife Amy and their infant daughter Cindy to escape from the impending danger to a secret location, donating his half of the money to her. Deciding to avenge Mary Ann's death by killing the remaining people involved, Welles returns to Hollywood, tracks down Poole, and takes him to the shooting location, but stops short of killing him. He calls Mrs. Mathews to tell her about her daughter and asks for her permission to punish those responsible. Mrs. Mathews breaks down once presented with the truth, but affirms that she loved her daughter and permits him to proceed. With that, Welles pistol whips Poole to death with his gun, burning his body and the pornography from his car. He then attacks Machine at his home and unmasks him, revealing a bespectacled man named George Anthony Higgins, who admits that his sadistic actions are simply done out of pleasure, and are unrelated to any personal trauma. Welles then kills him in the ensuing struggle. After returning to his family, Welles breaks down in front of Amy, attempting to process all the evil things that he had witnessed throughout the investigation. Months later, he receives a letter from Mrs. Mathews, thanking him and relating her gratitude for the fact that, despite everything, they were the only two people who really cared about Mary Ann.",
]
# summaries = df2["summary"]
# summaries = list(summaries)
scripts = df2["script"]
scripts = list(scripts)

In [9]:
# Encode the queries and documents. Note that queries benefit from using a prompt
# Here we use the prompt called "query" stored under `model.prompts`, but you can
# also pass your own prompt via the `prompt` argument
query_embeddings = model.encode(
    queries,
    prompt=MODELPROMPT,
)
# summaries_embeddings = model.encode(
#    summaries, batch_size=32, precision="float32", show_progress_bar=True
# )

scripts_embeddings = model.encode(
    scripts, batch_size=32, precision="float32", show_progress_bar=True
)

# Compute the (cosine) similarity between the query and document embeddings
similarity = model.similarity(query_embeddings, scripts_embeddings)
print(similarity)
# tensor([[0.7646, 0.1414],
#         [0.1355, 0.6000]])

Batches:   0%|          | 0/69 [00:03<?, ?it/s]


RuntimeError: Invalid buffer size: 4.00 GB

In [ ]:
model.prompts

In [ ]:
scripts_embeddings.shape

In [ ]:
scripts_embeddings.dump("scripts_embeddings")